<a href="https://colab.research.google.com/github/Ufuff/colab-ipynb/blob/main/Practice(spectrograms).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

данные сбалансированы, класс с шумом отсутствует

### Preprocessing

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:
import scipy.io.wavfile as wav
import matplotlib.pyplot as plt
import numpy.fft
from scipy.io import wavfile
import numpy as np

import xgboost as xgb
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import tree, metrics
from sklearn.tree import DecisionTreeClassifier
from skimage import color
from skimage import io
from __future__ import division, print_function
from os import listdir
import pandas as pd
import os
from time import time
from os.path import isfile, join
%matplotlib inline 


In [ ]:
!mkdir /spectrograms

In [ ]:
def wav_to_spectrogram(audio_path, save_path, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
    sample_rate, samples = wav.read(audio_path)

    fig = plt.figure()
    fig.set_size_inches((spectrogram_dimensions[0]/fig.get_dpi(), spectrogram_dimensions[1]/fig.get_dpi()))
    ax = plt.Axes(fig, [0., 0., 1., 1.])
    ax.set_axis_off()
    fig.add_axes(ax)
    ax.specgram(samples, cmap=cmap, Fs=2, noverlap=noverlap)
    ax.xaxis.set_major_locator(plt.NullLocator())
    ax.yaxis.set_major_locator(plt.NullLocator())
    fig.savefig(save_path, bbox_inches="tight", pad_inches=0)


def dir_to_spectrogram(audio_dir, spectrogram_dir, spectrogram_dimensions=(64, 64), noverlap=16, cmap='gray_r'):
    file_names = [f for f in listdir(audio_dir) if isfile(join(audio_dir, f)) and '.wav' in f]

    for file_name in file_names:
        print(file_name)
        audio_path = audio_dir + file_name
        spectogram_path = spectrogram_dir + file_name.replace('.wav', '.png')
        wav_to_spectrogram(audio_path, spectogram_path, spectrogram_dimensions=spectrogram_dimensions, noverlap=noverlap, cmap=cmap)


if __name__ == '__main__':
    audio_dir = "/content/drive/My Drive/recordings/"
    spectrogram_dir = "/spectrograms/"
    dir_to_spectrogram(audio_dir, spectrogram_dir)


In [ ]:
def get_arr(file):
  img = color.rgb2gray(io.imread(file))
  arr = np.array(img)
  arr = arr / 255
  return arr

In [ ]:
y_data = np.array([])
x_data = np.array([])

y_train = np.array([])
x_train = np.array([])
y_test = np.array([])
x_test = np.array([])

indir = '/spectrograms/'
for root, dirs, filenames in os.walk(indir):
    for f in filenames:
        y_data = np.append(y_data, f[0]) 
        
x_data = np.array([get_arr(indir+f) for f in filenames])

y_train = y_data[:1800]
x_train = x_data[:1800]
y_test = y_data[1800:]
x_test = x_data[1800:]

print(len(x_train))
print(y_train.size)
print(len(x_test))
print(y_test.size)

1800
1800
200
200


In [ ]:
y_train[:10]

array(['1', '0', '7', '4', '9', '9', '6', '9', '0', '2'], dtype='<U32')

In [ ]:
x_train = x_train.reshape(x_train.shape[0], 4096)
x_test = x_test.reshape(x_test.shape[0], 4096)

### LogisticRegression

In [ ]:
clf = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(x_train, y_train)
clf.score(x_train, y_train)
predictions = clf.predict(x_test)
metrics.accuracy_score(y_test, predictions)

0.535

### Decision Tree

In [ ]:
decision_tree = DecisionTreeClassifier(random_state=0, max_depth=10, min_samples_leaf = 3)
decision_tree = decision_tree.fit(x_train, y_train)
predictions = decision_tree.predict(x_test)
metrics.accuracy_score(y_test, predictions)

0.62

### Random Forest

In [ ]:
random_forest = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=1)
random_forest.fit(x_train,y_train)
predictions = random_forest.predict(x_test)
metrics.accuracy_score(y_test, predictions)

0.93

### Gradient Boosting

In [ ]:
g_boosting = xgb.XGBClassifier(learning_rate = 0.1, max_depth = 5, n_estimators = 150, min_child_weight = 3)
g_boosting.fit(x_train,y_train)
print(g_boosting.score(x_train, y_train))
print(g_boosting.score(x_test, y_test))

1.0
0.93


### Neural Networks

In [ ]:
import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPooling2D, Dropout, Flatten 
from tensorflow.keras import utils
from tensorflow.keras.preprocessing import image
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint
from tensorflow.keras import utils
import tensorflow as tf
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


In [ ]:
x_train = x_train.reshape(x_train.shape[0], 64, 64, 1)
x_test = x_test.reshape(x_test.shape[0], 64, 64, 1)
print(x_train.shape)
print(x_test.shape)

(1800, 64, 64, 1)
(200, 64, 64, 1)


In [ ]:
y_train = utils.to_categorical(y_train)
print(y_train[:5])
y_test = utils.to_categorical(y_test)

[[0. 1. 0. 0. 0. 0. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 1. 0. 0.]
 [0. 0. 0. 0. 1. 0. 0. 0. 0. 0.]
 [0. 0. 0. 0. 0. 0. 0. 0. 0. 1.]]


In [ ]:
random_seed = 2
X_train, X_val, Y_train, Y_val = train_test_split(x_train, y_train, test_size = 0.1, random_state=random_seed)

In [ ]:
model = Sequential()

model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu', input_shape = (64,64,1)))
model.add(Conv2D(filters = 32, kernel_size = (5,5),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.25))

model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(Conv2D(filters = 64, kernel_size = (3,3),padding = 'Same', 
                 activation ='relu'))
model.add(MaxPooling2D(pool_size=(2,2), strides=(2,2)))
model.add(Dropout(0.25))

model.add(Flatten())
model.add(Dense(256, activation = "relu"))
model.add(Dropout(0.5))
model.add(Dense(10, activation = "softmax"))

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


In [ ]:
model.compile(loss="categorical_crossentropy", optimizer="adam", metrics=["accuracy"])

print(model.summary())

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 64, 64, 32)        832       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 64, 64, 32)        25632     
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 32, 32, 32)        0         
_________________________________________________________________
dropout (Dropout)            (None, 32, 32, 32)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 32, 32, 64)        18496     
_________________________________________________________________
conv2d_3 (Conv2D)            (None, 32, 32, 64)        36928     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 16, 16, 64)        0

In [ ]:
сheckpoint = ModelCheckpoint('recognizer', 
                              monitor='val_acc', 
                              save_best_only=True,
                              verbose=1)

In [ ]:
learning_rate_reduction = ReduceLROnPlateau(monitor='val_acc', 
                                            patience=3, 
                                            verbose=1, 
                                            factor=0.5, 
                                            min_lr=0.00001)

In [ ]:
batch_size=96
history = model.fit(X_train,Y_train, batch_size=batch_size, 
                    epochs=15,
                    validation_data=(X_val, Y_val),
                    steps_per_epoch=X_train.shape[0] // batch_size,
                    verbose=1,
                    callbacks=[сheckpoint, learning_rate_reduction])

Train on 16 samples, validate on 180 samples
Epoch 1/15
15/16 [===========================>..] - ETA: 1s - loss: 2.2960 - acc: 0.1234
Epoch 00001: val_acc improved from -inf to 0.42222, saving model to recognizer
16/16 [==============================] - 27s 2s/step - loss: 2.2932 - acc: 0.1280 - val_loss: 2.2316 - val_acc: 0.4222
Epoch 2/15
15/16 [===========================>..] - ETA: 1s - loss: 1.7182 - acc: 0.4418
Epoch 00002: val_acc improved from 0.42222 to 0.77222, saving model to recognizer
16/16 [==============================] - 18s 1s/step - loss: 1.6800 - acc: 0.4522 - val_loss: 0.8328 - val_acc: 0.7722
Epoch 3/15
15/16 [===========================>..] - ETA: 1s - loss: 0.7395 - acc: 0.7410
Epoch 00003: val_acc improved from 0.77222 to 0.88889, saving model to recognizer
16/16 [==============================] - 18s 1s/step - loss: 0.7261 - acc: 0.7457 - val_loss: 0.3419 - val_acc: 0.8889
Epoch 4/15
15/16 [===========================>..] - ETA: 1s - loss: 0.4276 - acc: 0.8505

In [ ]:
scores = model.evaluate(x_test, y_test, verbose=1)

200/200 [==============================] - 0s 1ms/sample - loss: 0.0828 - acc: 0.9750
